In [1]:
import pandas as pd
import spot
import os
import re

In [2]:
tools_order = ['delag', 'rabinizer4', 'ltl2tgba', 'ltl3tela']

## Settings

In [3]:
os.environ['SPOT_TMPDIR'] = './autcross/'
os.environ['SPOT_TMPKEEP'] = '1'

In [4]:
timeout = 30

In [5]:
telatko = './telatko.py'

## Functions

In [6]:
def run_autcross(tool):
    !autcross --no-checks --csv=patterns-telatko-{tool}.csv -F patterns-{tool}.hoa 'autfilt %H > %O' '{telatko} -F %H -T {timeout} -L 1 -I -O %O' '{telatko} -F %H -T {timeout} -L 2 -I -O %O' '{telatko} -F %H -T {timeout} -L 3 -I -O %O' '{telatko} -F %H -T {timeout} -L 3 -I -G -O %O' > autcross/{tool}.log 2>&1

In [7]:
def get_data(tools_order):
    data = {}
    data_small = {}

    llre = re.compile('^Running \[A([1234])\].*\'(.*?)\'$')
    hnre = re.compile('^name: "(.*)"$')
    nxre = re.compile('_([TU])\\s*$')
    

    for tool in tools_order:
        csv = pd.read_csv('patterns-telatko-' + tool + '.csv')
        log = './autcross/' + tool + '.log'

        ht = {
    #        'aut_id': [],
            'L0.acc': [],
            'L1.acc': [],
            'L1.time': [],
            'L1.qbfr': [],
            'L2.acc': [],
            'L2.time': [],
            'L2.qbfr': [],
            'L3.acc': [],
            'L3.time': [],
            'L3.qbfr': [],
            'L4.acc': [],
            'L4.time': [],
            'L4.qbfr': [],
        }


        i = 0
        for _,r in csv.iterrows():
            ht['L' + str(i) + '.acc'].append(r['output.acc_sets'])
            if i > 0:
                ht['L' + str(i) + '.time'].append(r['time'])

            i = (i + 1) % 5

        logfd = open(log, 'r')
        for ll in logfd.readlines():
            lmatch = llre.match(ll)
            if lmatch:
                level = lmatch.group(1)
                hoa = lmatch.group(2)              
                

                hoafd = open(hoa, 'r')
                aut_seen = False
                name_seen = False
                for hl in hoafd.readlines():
                    
                    if len(hl) > 3:
                        aut_seen = True
                    hmatch = hnre.match(hl)
                    if hmatch:
                        name_seen = True
                        xmatch = nxre.search(hmatch.group(1))
                        if xmatch:
                            ht['L' + level + '.qbfr'].append(xmatch.group(1))
                        else:
                            ht['L' + level + '.qbfr'].append('U')
                        break

                hoafd.close()

                if not aut_seen:
                    ht['L' + level + '.qbfr'].append('X')
                elif not name_seen:
                    ht['L' + level + '.qbfr'].append('U')

        logfd.close()

        
        data_dd = pd.DataFrame(data = ht)
        data_de = data_dd.dropna()
        data_df = data_de[(data_de['L1.qbfr'] != 'X') & (data_de['L2.qbfr'] != 'X') & (data_de['L3.qbfr'] != 'X') & (data_de['L4.qbfr'] != 'X') & (data_de['L0.acc'] > 1)]
        data[tool] = data_df
    return data

In [8]:
def get_table(data,tools_order):
    sumdata = {
    'tool': [],
    'autcount': [],
    'L0.acc': [],
    }
    for l in range(1, 5):
        for s in ['acc', 'time', 'ratio', 'qbft', 'qbfu']:
            sumdata['L' + str(l) + '.' + s] = []

    for tool in tools_order:
        data_df = data[tool]

        sumdata['tool'].append(tool)
        sd = data_df.sum()
        #sumdata['ltlcount'].append(ltlf_translated[tool])
        sumdata['autcount'].append(len(data_df))
        for i in range(0, 5):
            sumdata['L' + str(i) + '.acc'].append(int(sd['L' + str(i) + '.acc']))
            if i > 0:
                t = sd['L' + str(i) + '.time']
                #if i > 1 and i < 4:
                #    t -= sd['L' + str(i - 1) + '.time']
                sumdata['L' + str(i) + '.time'].append(round(t, 1))
                sumdata['L' + str(i) + '.ratio'].append(
                    round(100 - 100 * int(sd['L' + str(i) + '.acc']) / int(sd['L0.acc']), 1)
                )

                for x in ['T', 'U']:
                    sumdata['L' + str(i) + '.qbf' + x.lower()].append(len(data_df[data_df['L' + str(i) + '.qbfr'] == x]))

    r = pd.DataFrame(data = sumdata)
    return r

## telatko

In [11]:
for tool in tools_order:
    run_autcross(tool)

^C
^C
^C
^C


In [12]:
aut_data = get_data(tools_order)

FileNotFoundError: [Errno 2] No such file or directory: 'patterns-telatko-delag.csv'

In [13]:
df = get_table(aut_data, tools_order)
df

NameError: name 'aut_data' is not defined